# ========================================================
# Dark URL Detection
# Adrien Manciet - Thibault Sourdeval
# ========================================================

In [ ]:
#Introduction

Ce dataset est un ensemble d'url qui sont labellisés. Si le label vaut 1, l'url est dangereux, si il vaut -1, il ne l'est pas. L'objectif sera de faire un algorithme de classification des url en apprenant sur le dataset disponible. 

# =========================
# Partie 1 : Phase d'exploration
# =========================

In [ ]:
#Code

Nous notons que les fichiers de données sont sous la forme de matrices sparse. Cela signifie que seules les valeurs non nulles sont gardées en mémoire. 
Cela permet d'épargner des erreurs de mémoire. 

Le fichier features contient des numéros de features qui sont moins souvent nuls que les autres. Ils pourraient aussi être des délimiteurs de groupe
de features appartenant à la même catégorie. 

Nous codons une fonction de prévisualisation pour mieux comprendre la structure des données en les transformant en un dataframe. Pour la suite, 
nous resterons dans le format de données initial.

**Fonction de prévisualisation** 

In [ ]:
from sklearn.datasets import load_svmlight_file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def preview_data(day, nb_lines, nb_cols, random = True):
    # Si random est laissé tel quel, une valeur au hasard 
    # est prise pour la première ligne et la première colonne à afficher

    path =f"url_svmlight/url_svmlight/Day{day}.svm"
    X, y = load_svmlight_file(path)

    if random == True:
        start_line = np.random.randint(0, len(y)-nb_lines)
        start_col = np.random.randint(0,X.shape[1])
    else : 
        start_line = int(input("Première ligne à afficher : "))
        start_col = int(input("Première colonne à afficher : "))
    
    label_list = []
    for i in range(start_col, start_col+nb_cols):
        label_list.append(i)

    X_df = pd.DataFrame(X[start_line: start_line+nb_lines, start_col: start_col+nb_cols].toarray(), columns=label_list)
    y_df = pd.DataFrame(y[start_line: start_line+nb_lines], columns=['label'])

    data = pd.concat([X_df, y_df], axis=1)
    return data

# preview_data(17, 10, 10, random=False)


**Visualisations grahiques** 

In [ ]:
def scatter_plot(day, feature_x, feature_y): 
    path =f"url_svmlight/url_svmlight/Day{day}.svm"
    X, y = load_svmlight_file(path)

    X = X[0:100,:100].toarray()
    plt.figure()
    plt.grid(alpha=0.2)
    sc = plt.scatter(X[:,feature_x], X[:,feature_y], c=y[0:100], cmap="viridis")
    plt.xlabel(f'Feature {feature_x}')
    plt.ylabel(f'Feature {feature_y}')
    

    cbar = plt.colorbar(sc)
    cbar.set_label("Label")
    plt.show()

scatter_plot(22,4,3)

# =========================
# Partie 2 : Feature Engineering
# =========================

In [ ]:
#Code



Nous commençons par une pca brute pour voir ce que cela peut donner. 

**PCA**

In [ ]:
from sklearn.decomposition import TruncatedSVD

X, y = load_svmlight_file('url_svmlight/url_svmlight/Day96.svm')

svd = TruncatedSVD(n_components=10)
X_svd = svd.fit_transform(X)
# print(X_svd.shape)


In [ ]:
# svd.explained_variance_ratio_
plt.scatter(X_svd[:1000,0], X_svd[:1000,1], c=y[:1000], cmap='coolwarm', s=10, alpha=0.6)
plt.xlabel('Composante 1')
plt.ylabel('Composante 2')
plt.title('Projection des URLs sur les 2 premières composantes SVD')
plt.colorbar(label='label')
plt.show()

# =========================
# Partie 3 : Phase d'apprentissage
# =========================

In [ ]:
#Code

# =========================
# Partie 4 : Tuning d'un hyperparamètre
# =========================

In [ ]:
#Code

# =========================
# Partie 5 : Conclusions
# =========================

In [ ]:
#Code